In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df = pd.read_csv("../data/Churn_Modelling.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
col_to_drop = ['RowNumber', 'CustomerId', 'Surname']

In [5]:
df = df.drop(col_to_drop, axis=1)

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(drop=True, inplace=True)

In [8]:
cat_cols = ['Geography', 'Gender', 'HasCrCard','IsActiveMember']
con_cols = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
y_col = ['Exited']

In [9]:
cats = np.stack([df[cat].astype('category').cat.codes.values for cat in cat_cols], axis=1)

In [10]:
cats_size = [len(df[cat].astype('category').cat.categories) for cat in cat_cols]

In [11]:
cons = np.stack([df[con].values for con in con_cols], axis=1)

In [12]:
y = df[y_col].values.reshape(-1,1)

In [13]:
emb_sizes = [(size, (size+1)//2) for size in cats_size]

In [14]:
class TabularModel(nn.Module):
    def __init__(self, emb_sizes, no_con_cols,layers, out_size, p):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni,nf) for ni, nf in emb_sizes])
        self.drop = nn.Dropout(p)
        self.batch_normal = nn.BatchNorm1d(no_con_cols)
        
        layer_list = []
        
        no_cat_cols = sum([size for _, size in emb_sizes])
        no_input = no_cat_cols + no_con_cols
        
        for layer in layers:
            layer_list.append(nn.Linear(no_input, layer))
            layer_list.append(nn.ReLU(inplace=True))
            layer_list.append(nn.BatchNorm1d(layer))
            layer_list.append(nn.Dropout(p))
            
            no_input = layer
            
        layer_list.append(nn.Linear(layers[-1], out_size))
        layer_list.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layer_list)
        
    def forward(self, x_cats, x_cons):
        embeddings = []
        for i, emb in enumerate(self.embeds):
            embeddings.append(emb(x_cats[:,i]))
        
        x_cats = torch.cat(embeddings, axis=1)
        x_cats = self.drop(x_cats)
        
        x_cons = self.batch_normal(x_cons)
        
        x = torch.cat([x_cats,x_cons], axis=1)
        
        x = self.layers(x)
        return x

In [15]:
net = TabularModel(emb_sizes,cons.shape[1], [100,50,25], 1, .4)
CUDA = torch.cuda.is_available()

In [16]:
batch_size = len(df)
test_size = int(len(df) * .2)

In [17]:
cons_train = torch.FloatTensor(cons[:batch_size - test_size])
cons_test = torch.FloatTensor(cons[batch_size - test_size : batch_size])
cats_train = torch.LongTensor(cats[:batch_size - test_size])
cats_test = torch.LongTensor(cats[batch_size - test_size: batch_size])
y_train = torch.FloatTensor(y[:batch_size - test_size])
y_test = torch.FloatTensor(y[batch_size - test_size: batch_size])

In [18]:
datasets = TensorDataset(cons_train, cats_train, y_train)

In [19]:
data_loader = DataLoader(datasets, batch_size=200)

In [20]:
criterion = nn.HingeEmbeddingLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.002)

In [21]:
epochs = 500
losses = []

for epoch in range(epochs):
    y_p = net(cats_train, cons_train)
    loss = criterion(y_p, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss)
    if (epoch + 1) % 20 == 0:
        print('Epoch [{}/{}]. Loss : {}'.format(epoch+1, epochs, loss))

Epoch [20/500]. Loss : 0.8425856828689575
Epoch [40/500]. Loss : 0.8224560022354126
Epoch [60/500]. Loss : 0.8142885565757751
Epoch [80/500]. Loss : 0.8092086315155029
Epoch [100/500]. Loss : 0.8059837818145752


KeyboardInterrupt: 

In [ ]:
plt.plot(range(epochs),losses)

In [ ]:
with torch.no_grad():
    y_pred = net(cats_test, cons_test)
    loss = criterion(y_pred, y_test)
    print(loss)

In [ ]:
y_pred.argmax(axis=1)

In [52]:
y_test_to_int = y_test.type(torch.int)

In [53]:
true_val = y_test_to_int.flatten()

In [54]:
((y_pred.argmax(axis=1) == true_val)+0).type(torch.float).mean()

tensor(0.7950)